# メトリクスを使った学習ジョブの監視
本ハンズオンではAmazon SageMaker を使用して学習した場合のメトリクスの監視について実施します。 前半では Amazon CloudWatch を使用した監視を行います。CloudWatch では、学習ジョブから出力された raw データを収集し、ほぼリアルタイムの読み取り可能なメトリクスに加工できます。

また後半では、Keras がバッチ毎に出力した学習ログを活用して TensorBoard にて学習の状況を監視する方法を実践します。

## Amazon SageMaker Python SDK での学習メトリクスの定義
CloudWatch にメトリクスを送信するためには、標準出力を正規表現でパースし、Estimator の metric_definitions 引数に渡します。もし`train:error`と`validation:error`というメトリクスを監視したい場合には、下記のようになります。

```python 
estimator = Estimator(image_name=ImageName,
            role='SageMakerRole', train_instance_count=1,
            train_instance_type='ml.c4.xlarge',            
            k=10,
            metric_definitions=[
                   {'Name': 'train:error', 'Regex': 'Train_error=(.*?);'},
                   {'Name': 'validation:error', 'Regex': 'Valid_error=(.*?);'}
            ])
```

## cifar10 の学習ジョブの監視
今回の Keras のがどのようなメトリクスを出力しているか、以前の cifar10_keras_sm の学習ジョブを SageMaker のコンソールから探しましょう。 学習ジョブの詳細からモニタリング、ログの表示と進んでいくことで標準出力を確認することが出来ます。

確認したメトリクスをパースできるように metric_definitions を定義します。

In [1]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

`metric_definitions=metric_definitions` として、学習ジョブを実行します。  

In [3]:
import os
import sagemaker
from sagemaker import get_execution_role

from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()
role = get_execution_role()

# dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')

In [4]:
estimator = TensorFlow(base_job_name='cifar10-cloudwatch',
                       entry_point='cifar10_keras_sm.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions)


estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2019-10-01 05:56:25 Starting - Starting the training job...
2019-10-01 05:56:27 Starting - Launching requested ML instances......
2019-10-01 05:57:29 Starting - Preparing the instances for training......
2019-10-01 05:58:42 Downloading - Downloading input data...
2019-10-01 05:59:11 Training - Downloading the training image..2019-10-01 05:59:33,497 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-10-01 05:59:33,896 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "model_dir": "s3://sagemaker-us-east-1-815969174475/cifar10-2019-10-01-05

### View the job training metrics
学習中のメトリクスの遷移は [SageMaker のコンソール](https://console.aws.amazon.com/sagemaker/home) から確認することが出来ます。下記のセルに表示されるリンクもお使い下さい。

In [8]:
from IPython.core.display import Markdown

link = 'https://console.aws.amazon.com/cloudwatch/home?region='+sagemaker_session.boto_region_name+'#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20'+estimator.latest_training_job.job_name
display(Markdown('Cloud Watch を確認される際にはこちらのリンクをお使い下さい。: [CloudWatch リンク]('+link+')'))

Cloud Watch を確認される際にはこちらのリンクをお使い下さい。: [CloudWatch リンク](https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20cifar10-2019-10-01-05-56-25-075)

## TensorBoard を使った監視
次は TensorBoard を使ったメトリクスの監視を行います。

`cifar10_keras_sm.py` をアップデートして 学習ログを TensorBoard へ送信しましょう。変更点は下記2点です。

①`from keras.callbacks import TensorBoard` の追加

② ModelCheckpoint callback の後に TensorBoard callback の追加。

```python
callbacks.append(TensorBoard(log_dir=args.model_dir,update_freq='epoch'))
```

### TensorBoard を使った学習ジョブの実行

In [9]:
estimator = TensorFlow(base_job_name='cifar10-tensorboard',
                       entry_point='cifar10_keras_sm.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions)

In [ ]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2019-10-01 06:05:13 Starting - Starting the training job...
2019-10-01 06:05:14 Starting - Launching requested ML instances......
2019-10-01 06:06:20 Starting - Preparing the instances for training......
2019-10-01 06:07:36 Downloading - Downloading input data...
2019-10-01 06:08:14 Training - Downloading the training image...
2019-10-01 06:08:39 Training - Training image download completed. Training in progress..2019-10-01 06:08:42,739 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-10-01 06:08:43,228 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparame

TensorBoard で実行された学習ジョブを確認します。まずは [TensorBoard](https://github.com/tensorflow/tensorboard) をインストールします。

In [12]:
!pip install tensorboard

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


次に TensorBoard をノートブックインスタンス上で起動します。お使いのリージョンと、学習ログを保管したS3パスから、起動コマンドを作成します。

In [13]:
from IPython.core.display import Markdown

link = 'AWS_REGION=\''+sagemaker_session.boto_region_name+'\' tensorboard --logdir ' + estimator.model_dir + ' --host localhost --port 6006'
display(Markdown('表示された TensorBoard の起動コマンドを次のセルで実行して下さい'))
display(Markdown('!'+link))

表示された TensorBoard の起動コマンドを次のセルで実行して下さい

!AWS_REGION='us-east-1' tensorboard --logdir s3://sagemaker-us-east-1-815969174475/cifar10-tensorboard-2019-10-01-06-05-12-953/model --host localhost --port 6006

In [14]:
!AWS_REGION='us-east-1' tensorboard --logdir s3://sagemaker-us-east-1-815969174475/cifar10-tensorboard-2019-10-01-06-05-12-953/model --host localhost --port 6006

TensorBoard 1.14.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


下記のリンクをご自身のお使いの値に合わせて変更し、ブラウザでアクセスして下さい。\
`https://https://ノートブックインスタンス名.notebook.お使いのリージョン名.sagemaker.aws/proxy/6006/`